A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
function [f,s,m,mv,mvd,unA] = ...
          expmv(t,A,b,M,prec,shift,bal,full_term,prnt)
%EXPMV   Matrix exponential times vector or matrix.
%   [F,S,M,MV,MVD] = EXPMV(t,A,B,[],PREC) computes EXPM(t*A)*B without
%   explicitly forming EXPM(t*A). PREC is the required accuracy, 'double',
%   'single' or 'half', and defaults to CLASS(A).
%   A total of MV products with A or A^* are used, of which MVD are
%   for norm estimation.
%   The full syntax is
%     [f,s,m,mv,mvd,unA] = expmv(t,A,b,M,prec,shift,bal,full_term,prnt).
%   unA = 1 if the alpha_p were used instead of norm(A).
%   If repeated invocation of EXPMV is required for several values of t
%   or B, it is recommended to provide M as an external parameter as
%   M = SELECT_TAYLOR_DEGREE(A,b,m_max,p_max,prec,shift,bal,true).
%   This also allows choosing different m_max and p_max.

%   Reference: A. H. Al-Mohy and N. J. Higham. Computing the action of the
%   matrix exponential, with an application to exponential integrators.
%   SIAM J. Sci. Comput., 33(2):488--511, 2011.  Algorithm 3.2.

%   Awad H. Al-Mohy and Nicholas J. Higham, November 9, 2010.

if nargin < 9 || isempty(prnt), prnt = false; end
if nargin < 8 || isempty(full_term), full_term = false; end
if nargin < 7 || isempty(bal), bal = false; end
if bal
    [D,B] = balance(A);
    if norm(B,1) < norm(A,1), A = B; b = D\b; else bal = false; end
end

if nargin < 6 || isempty(shift), shift = true; end
n = length(A);
if shift
    mu = trace(A)/n; mu = full(mu); % Much slower without the full!
    A = A-mu*speye(n);
end

if nargin < 5 || isempty(prec), prec = class(A); end
if nargin < 4 || isempty(M)
   tt = 1;
   [M,mvd,alpha,unA] = select_taylor_degree(t*A,b,[],[],prec,false,false);
   mv = mvd;
else
   tt = t; mv = 0; mvd = 0;
end

switch prec
    case 'double', tol = 2^(-53);
    case 'single', tol = 2^(-24);
    case 'half',   tol = 2^(-10);
end

s = 1;
if t == 0
    m = 0;
else
    [m_max,p] = size(M);
     U = diag(1:m_max);
     C = ( (ceil(abs(tt)*M))'*U );
     C (C == 0) = inf;
     if p > 1
         [cost m] = min(min(C)); % cost is the overall cost.
     else
         [cost m] = min(C);  % when C is one column. Happens if p_max = 2.
     end
     if cost == inf; cost = 0; end
     s = max(cost/m,1);
end
eta = 1;
if shift, eta = exp(t*mu/s); end
f = b;
if prnt, fprintf('m = %2.0f, s = %g, m_actual = ', m, s), end
for i = 1:s
    c1 = norm(b,inf);
    for k = 1:m

        b = (t/(s*k))*(A*b);
        mv = mv + 1;
        f =  f + b;
        c2 = norm(b,inf);
        if ~full_term
%            if prnt, fprintf(' %9.2e, \n', (c1+c2)/norm(f,inf)), pause, end
            if c1 + c2 <= tol*norm(f,inf)
                if prnt, fprintf(' %2.0f, ', k), end
                break
            end
            c1 = c2;
        end

    end
    f = eta*f; b = f;
end
if prnt, fprintf('\n'), end
if bal, f = D*f; end


In [ ]:
%example_WTI_orig20181206.m

% % The Stochastic Volatility Inspired parameterization of the Implied
% % Volatility Smile
% %% raw SVI parameterization
% a = -0.041;
% b = 0.1331;
% m = 0.3586;
% rho = 0.3060;
% sigma = 0.4153;
% param_raw = [a; b; m; rho; sigma];
% 
% k = -1.5:0.01:1.5;
% w_r = svi_raw(k,param_raw);
% 
% plot(k,w_r, 'ob');
% 
% %% natural SVI parameterization
% 
% param_natural = svi_convertparameters( param_raw, 'raw', 'natural');
% w_n = svi_natural(k,param_natural);
% 
% plot(k,w_r, 'ob');
% hold on
% plot(k,w_n, 'xr');
% legend('Raw parameterization','Natural parameterization');
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_n-w_r)>1e-10));
% 
% %% Jump-Wings parameterization
% 
% tau = 1;
% param_jw = svi_convertparameters( param_raw, 'raw', 'jumpwing', tau);
% w_j = svi_jumpwing(k,param_jw, tau);
% 
% plot(k,w_r, 'ob');
% hold on
% plot(k,w_j, 'xr');
% legend('Raw parameterization','Jump-Wing parameterization');
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_j-w_r)>1e-10));
% 
% %% Test conversion in other direction
% v = 0.01742625;
% psi = 0.1752111;
% p = 0.6997381;
% c = 1.316798;
% vt = 0.0116249;
% param_jw = [v; psi; p; c; vt];
% param_raw = svi_convertparameters( param_jw, 'jumpwing', 'raw', tau);
% param_natural = svi_convertparameters( param_jw, 'jumpwing', 'natural', tau);
% 
% w_r = svi_raw(k,param_raw);
% w_n = svi_natural(k,param_natural);
% w_j = svi_jumpwing(k,param_jw, tau);
% 
% plot(k, w_r, 'ob')
% hold on
% plot(k, w_n, 'xr')
% plot(k, w_j, 'xg')
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_n-w_r)>1e-10));
% display(any(abs(w_j-w_r)>1e-10));
% display(any(abs(w_j-w_n)>1e-10));
% 
% %% Arbitrage-free Surface SVI
% theta = 0.1:0.1:1;
% rho = 0.3;
% phifun = 'heston_like';
% phi_param = (1+abs(rho))/4 + 1;
% 
% [totalvariance] = svi_surface(k, theta, rho, phifun, phi_param);
% plot(k, totalvariance)
% 
% phifun = @(theta, lambda) 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
% param_natural = [0; 0; rho; theta(4); phifun(theta(4),phi_param)];
% totalvariance_nat = svi_natural(k,param_natural);
% 
% plot(k, totalvariance_nat, 'ob')
% hold on
% plot(k, totalvariance(:,4), 'xr');
% hold off
% 
% display(any(abs(totalvariance_nat-totalvariance(:,4)) > 1e-10));

%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

load('WTIForward.mat','WTIForward')
load('WTIexpiry.mat','WTIexpiry')
load('WTIPremiums.mat','WTIPremiums')
load('WTIStrikes.mat','WTIStrikes')
load('WTIrate.mat','WTIrate')

valueDate = '2018-12-06';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-02-14'),H_Date('2019-03-15'),H_Date('2019-05-16'),H_Date('2019-11-15'),...
                      H_Date('2020-06-17'),H_Date('2020-12-16')};

optionMaturity = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity)
    optionMaturity(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity6 = optionMaturity6/365.0;
maturityLong = repelem(maturity6,55);

strike6 = [];
forward6 = [53.67	53.9	54.32	54.7	54.66	54.59];
rate6 = [0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];
reateLong = repelem(rate,55);
forwardLong = repelem(forward6,55);
premium = 0.0;

% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
mid = WTIPremiums;


forward = WTIForward;
maturity = WTIexpiry;

pos_remove = mid < 0.02 |  isnan(mid);

% pos_remove = isnan(WTIPremiums);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = WTIrate(~pos_remove);
strike = WTIStrikes(~pos_remove);
maturity = maturity(~pos_remove);
% clear close pos_remove

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
bloombergForwardMoneyness = [0.770411776	0.855897149	0.917477175	0.969945966	1.018203838	1.06497112	1.113322154	1.169032979	1.249506242
0.748552876	0.841539889	0.90987013	0.968831169	1.023692022	1.07755102	1.133988868	1.199721707	1.295454545
0.720305596	0.822772459	0.900257732	0.968464654	1.033081738	1.097790869	1.166439617	1.245360825	1.362242268
0.671096892	0.791389397	0.886032907	0.971517367	1.053345521	1.135886654	1.224771481	1.328957952	1.485411335
0.649012075	0.777296012	0.880717161	0.974698134	1.06381266	1.152140505	1.246121478	1.359220637	1.553311379
0.633632533	0.767594798	0.877614948	0.97713867	1.070727239	1.163418208	1.263454845	1.387378641	1.592855834
];

bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));

for t=1:T
    params.idx = t;
    for idxhh=1:length(bb_forwardMoneyness1)
        bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
        bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
    end
end


cc = 0.0;
aa =  findTargetStrike(-0.1,params);
bb = 0;
function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end


% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% 
% bbb= 0.0;
% 
% %% market data for WTI
% load('wtiFutures.mat','wtiFutures');
% load('wtiLogMoneyness.mat','wtiLogMoneyness');
% load('wtiMarketVol.mat','wtiMarketVol');
% load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');
% 
% implied_volatility = wtiMarketVol;
% maturity = wtiMaturity;
% log_moneyness = wtiLogMoneyness;
% 
% 
% % plot total implied volatility
% total_implied_volatility = implied_volatility.^2 .* maturity;
% maturities = sort(unique(maturity));
% T = length(maturities);
% figure
% hold on
% for t = 1:T
%     pos = maturity==maturities(t);
%     [x, idx] = sort(log_moneyness(pos));
%     y = total_implied_volatility(pos);
%     y = y(idx);
%     plot(x,y);
% end
% hold off
% 
% % fit SSVI to data
% phifun = 'power_law';
% parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % get fitted total implied variances
% total_implied_variance = implied_volatility.^2 .* maturity;
% model_total_implied_variance = zeros(size(total_implied_variance));
% model_implied_volatility = zeros(size(total_implied_variance));
% for t = 1:T
%     pos = maturity==maturities(t);
%     [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
%         svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
% end
% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% aaa = 1.0;
% 
% %% inter-/ extrapolation of SVI fit
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear ask bid put
% 
% % estimate implied volatility
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% % implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
% for idxhh =1:length(mid)
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end
% 
% % fit SSVI to data
% phifun = 'power_law';
% [parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % load('SVIparameters.mat','parameters');
% % load('SVItheta.mat','theta');
% % load('SVImaturities.mat','maturities');
% 
% 
% log_moneyness = -1.5:0.1:1.5;
% 
% [~, idx] = ismember(maturities, maturity);
% forward_theta = forward(idx);
% interestrate_theta = interest_rate(idx);
% 
% tau_interp = 30/365.25;
% forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
% interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');
% 
% 
% [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
%     forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);
% 
% strike = forward_theta*exp(log_moneyness);
% figure
% plot(strike, call_price);



In [ ]:
%example_WTI_orig20181204_teNorCorrected.m

%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

load('WTIPremiums1204.mat', 'WTIPremiums1204')

valueDate = '2018-12-04';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-01-16'),H_Date('2019-02-14'),H_Date('2019-03-15'),H_Date('2019-05-16'),H_Date('2019-11-15'),...
                      H_Date('2020-05-14'),H_Date('2020-11-17')};

optionMaturity7 = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity7)
    optionMaturity7(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity7 = optionMaturity7/365.0;

strike7 = [21:1:75];
forward7 = [53.46	53.67	53.9	54.32	54.7	54.66	54.59];
rate7 = [0.02209	0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];

maturitySize = length(maturity7);
strikeSize = length(strike7);
maturityLong = repelem(maturity7,strikeSize)';
strikeLong = repmat(strike7,1,maturitySize)';
rateLong = repelem(rate7,strikeSize)';
forwardLong = repelem(forward7,strikeSize)';


maturity = maturityLong;
strike = strikeLong;
rate = rateLong;
forward = forwardLong;

mid = WTIPremiums1204;

pos_remove = mid < 0.02 |  isnan(mid);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = rate(~pos_remove);
strike = strike(~pos_remove);
maturity = maturity(~pos_remove);

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

% deltaMoneyness vol Surface -0.1, -0.15,...

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
% bloombergForwardMoneyness = [0.770411776	0.855897149	0.917477175	0.969945966	1.018203838	1.06497112	1.113322154	1.169032979	1.249506242
% 0.748552876	0.841539889	0.90987013	0.968831169	1.023692022	1.07755102	1.133988868	1.199721707	1.295454545
% 0.720305596	0.822772459	0.900257732	0.968464654	1.033081738	1.097790869	1.166439617	1.245360825	1.362242268
% 0.671096892	0.791389397	0.886032907	0.971517367	1.053345521	1.135886654	1.224771481	1.328957952	1.485411335
% 0.649012075	0.777296012	0.880717161	0.974698134	1.06381266	1.152140505	1.246121478	1.359220637	1.553311379
% 0.633632533	0.767594798	0.877614948	0.97713867	1.070727239	1.163418208	1.263454845	1.387378641	1.592855834
% ];
% 
% bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
% bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
% 
% for t=1:T
%     params.idx = t;
%     for idxhh=1:length(bb_forwardMoneyness1)
%         bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
%         bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
%     end
% end

disp('finished!')

function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end




In [ ]:

%example_WTI_orig20181204.m

%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

load('WTIForward.mat','WTIForward')
load('WTIexpiry.mat','WTIexpiry')
load('WTIPremiums.mat','WTIPremiums')
load('WTIStrikes.mat','WTIStrikes')
load('WTIrate.mat','WTIrate')
load('WTIPremiums1204.mat', 'WTIPremiums1204')
% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
valueDate = '2018-12-04';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-01-16'),H_Date('2019-02-14'),H_Date('2019-03-15'),H_Date('2019-05-16'),H_Date('2019-11-15'),...
                      H_Date('2020-05-14'),H_Date('2020-11-17')};

optionMaturity7 = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity7)
    optionMaturity7(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity7 = optionMaturity7/365.0;
% 55 is the size of the strike 21~ 75
maturityLong = repelem(maturity7,55);

strike7 = [];
forward7 = [53.67	53.9	54.32	54.7	54.66	54.59];
rate7 = [0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];
reateLong = repelem(rate,55);
forwardLong = repelem(forward6,55);


forward = WTIForward;
maturity = WTIexpiry;

pos_remove = mid < 0.02 |  isnan(mid);

% pos_remove = isnan(WTIPremiums);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = WTIrate(~pos_remove);
strike = WTIStrikes(~pos_remove);
maturity = maturity(~pos_remove);
% clear close pos_remove

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
bloombergForwardMoneyness = [0.770411776	0.855897149	0.917477175	0.969945966	1.018203838	1.06497112	1.113322154	1.169032979	1.249506242
0.748552876	0.841539889	0.90987013	0.968831169	1.023692022	1.07755102	1.133988868	1.199721707	1.295454545
0.720305596	0.822772459	0.900257732	0.968464654	1.033081738	1.097790869	1.166439617	1.245360825	1.362242268
0.671096892	0.791389397	0.886032907	0.971517367	1.053345521	1.135886654	1.224771481	1.328957952	1.485411335
0.649012075	0.777296012	0.880717161	0.974698134	1.06381266	1.152140505	1.246121478	1.359220637	1.553311379
0.633632533	0.767594798	0.877614948	0.97713867	1.070727239	1.163418208	1.263454845	1.387378641	1.592855834
];

bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));

for t=1:T
    params.idx = t;
    for idxhh=1:length(bb_forwardMoneyness1)
        bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
        bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
    end
end


cc = 0.0;
aa =  findTargetStrike(-0.1,params);
bb = 0;
function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end


% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% 
% bbb= 0.0;
% 
% %% market data for WTI
% load('wtiFutures.mat','wtiFutures');
% load('wtiLogMoneyness.mat','wtiLogMoneyness');
% load('wtiMarketVol.mat','wtiMarketVol');
% load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');
% 
% implied_volatility = wtiMarketVol;
% maturity = wtiMaturity;
% log_moneyness = wtiLogMoneyness;
% 
% 
% % plot total implied volatility
% total_implied_volatility = implied_volatility.^2 .* maturity;
% maturities = sort(unique(maturity));
% T = length(maturities);
% figure
% hold on
% for t = 1:T
%     pos = maturity==maturities(t);
%     [x, idx] = sort(log_moneyness(pos));
%     y = total_implied_volatility(pos);
%     y = y(idx);
%     plot(x,y);
% end
% hold off
% 
% % fit SSVI to data
% phifun = 'power_law';
% parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % get fitted total implied variances
% total_implied_variance = implied_volatility.^2 .* maturity;
% model_total_implied_variance = zeros(size(total_implied_variance));
% model_implied_volatility = zeros(size(total_implied_variance));
% for t = 1:T
%     pos = maturity==maturities(t);
%     [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
%         svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
% end
% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% aaa = 1.0;
% 
% %% inter-/ extrapolation of SVI fit
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear ask bid put
% 
% % estimate implied volatility
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% % implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
% for idxhh =1:length(mid)
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end
% 
% % fit SSVI to data
% phifun = 'power_law';
% [parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % load('SVIparameters.mat','parameters');
% % load('SVItheta.mat','theta');
% % load('SVImaturities.mat','maturities');
% 
% 
% log_moneyness = -1.5:0.1:1.5;
% 
% [~, idx] = ismember(maturities, maturity);
% forward_theta = forward(idx);
% interestrate_theta = interest_rate(idx);
% 
% tau_interp = 30/365.25;
% forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
% interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');
% 
% 
% [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
%     forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);
% 
% strike = forward_theta*exp(log_moneyness);
% figure
% plot(strike, call_price);



In [ ]:
%example_WTI_orig.m

% The Stochastic Volatility Inspired parameterization of the Implied
% Volatility Smile
%% raw SVI parameterization
a = -0.041;
b = 0.1331;
m = 0.3586;
rho = 0.3060;
sigma = 0.4153;
param_raw = [a; b; m; rho; sigma];

k = -1.5:0.01:1.5;
w_r = svi_raw(k,param_raw);

plot(k,w_r, 'ob');

%% natural SVI parameterization

param_natural = svi_convertparameters( param_raw, 'raw', 'natural');
w_n = svi_natural(k,param_natural);

plot(k,w_r, 'ob');
hold on
plot(k,w_n, 'xr');
legend('Raw parameterization','Natural parameterization');
hold off

% check that results are equivalent
display(any(abs(w_n-w_r)>1e-10));

%% Jump-Wings parameterization

tau = 1;
param_jw = svi_convertparameters( param_raw, 'raw', 'jumpwing', tau);
w_j = svi_jumpwing(k,param_jw, tau);

plot(k,w_r, 'ob');
hold on
plot(k,w_j, 'xr');
legend('Raw parameterization','Jump-Wing parameterization');
hold off

% check that results are equivalent
display(any(abs(w_j-w_r)>1e-10));

%% Test conversion in other direction
v = 0.01742625;
psi = 0.1752111;
p = 0.6997381;
c = 1.316798;
vt = 0.0116249;
param_jw = [v; psi; p; c; vt];
param_raw = svi_convertparameters( param_jw, 'jumpwing', 'raw', tau);
param_natural = svi_convertparameters( param_jw, 'jumpwing', 'natural', tau);

w_r = svi_raw(k,param_raw);
w_n = svi_natural(k,param_natural);
w_j = svi_jumpwing(k,param_jw, tau);

plot(k, w_r, 'ob')
hold on
plot(k, w_n, 'xr')
plot(k, w_j, 'xg')
hold off

% check that results are equivalent
display(any(abs(w_n-w_r)>1e-10));
display(any(abs(w_j-w_r)>1e-10));
display(any(abs(w_j-w_n)>1e-10));

%% Arbitrage-free Surface SVI
theta = 0.1:0.1:1;
rho = 0.3;
phifun = 'heston_like';
phi_param = (1+abs(rho))/4 + 1;

[totalvariance] = svi_surface(k, theta, rho, phifun, phi_param);
plot(k, totalvariance)

phifun = @(theta, lambda) 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
param_natural = [0; 0; rho; theta(4); phifun(theta(4),phi_param)];
totalvariance_nat = svi_natural(k,param_natural);

plot(k, totalvariance_nat, 'ob')
hold on
plot(k, totalvariance(:,4), 'xr');
hold off

display(any(abs(totalvariance_nat-totalvariance(:,4)) > 1e-10));

%% calibrate SSVI to Sample Data
clear
load sampledata
% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
forward = close.*exp((interest_rate-dividend_yield).*maturity);
pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
ask = ask(~pos_remove);
bid = bid(~pos_remove);
dividend_yield = dividend_yield(~pos_remove);
forward = forward(~pos_remove);
interest_rate = interest_rate(~pos_remove);
maturity = maturity(~pos_remove);
put = put(~pos_remove);
strike = strike(~pos_remove);
clear close pos_remove

% prepare data
bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
mid = (bid+ask)/2;
log_moneyness = log(strike./forward);
clear put

% estimate implied volatility
% implied_volatility_bid = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, bid);
% implied_volatility_ask = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, ask);
% implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);

implied_volatility_bid = zeros(length(bid),1);
implied_volatility_ask = zeros(length(bid),1);
implied_volatility     = zeros(length(bid),1);

for idxhh =1:length(bid)
    implied_volatility_bid(idxhh) = blackVolLBR(bid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
    implied_volatility_ask(idxhh) = blackVolLBR(ask(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
end

%% market data for WTI
load('wtiFutures.mat','wtiFutures');
load('wtiLogMoneyness.mat','wtiLogMoneyness');
load('wtiMarketVol.mat','wtiMarketVol');
load('wtiMaturity.mat','wtiMaturity');
load('wtiStrikes.mat','wtiStrikes');

implied_volatility = wtiMarketVol;
maturity = wtiMaturity;
log_moneyness = wtiLogMoneyness;


% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

% plot fitted data versus market data
for t = 1:T
    pos = maturity==maturities(t);
    
    subplot(3,5,t);
    plot(log_moneyness(pos), implied_volatility(pos), 'xb');
    hold on
    plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
    hold off
end

aaa = 1.0;

%% inter-/ extrapolation of SVI fit
clear
load sampledata
% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
forward = close.*exp((interest_rate-dividend_yield).*maturity);
pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
ask = ask(~pos_remove);
bid = bid(~pos_remove);
dividend_yield = dividend_yield(~pos_remove);
forward = forward(~pos_remove);
interest_rate = interest_rate(~pos_remove);
maturity = maturity(~pos_remove);
put = put(~pos_remove);
strike = strike(~pos_remove);
clear close pos_remove

% prepare data
bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
mid = (bid+ask)/2;
log_moneyness = log(strike./forward);
clear ask bid put

% estimate implied volatility
% implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
end

% fit SSVI to data
phifun = 'power_law';
[parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% load('SVIparameters.mat','parameters');
% load('SVItheta.mat','theta');
% load('SVImaturities.mat','maturities');


log_moneyness = -1.5:0.1:1.5;

[~, idx] = ismember(maturities, maturity);
forward_theta = forward(idx);
interestrate_theta = interest_rate(idx);

tau_interp = 30/365.25;
forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');


[call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
    forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);

strike = forward_theta*exp(log_moneyness);
figure
plot(strike, call_price);



In [ ]:
%example_WTI_20181113.m

%% market data for WTI
clc;
clear;

load('wtiFutures.mat','wtiFutures');
load('wtiLogMoneyness.mat','wtiLogMoneyness');
load('wtiMarketVol.mat','wtiMarketVol');
load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');

load('wtiLogMoneyness13.mat','wtiLogMoneyness13');
load('wtiMarketVol13.mat','wtiMarketVol13');
load('wtiMaturity13.mat','wtiMaturity13');

load('wtiMarketVol50150.mat','wtiMarketVol50150');
load('wtiLogMoneyness50150.mat','wtiLogMoneyness50150');
load('maturity50150.mat','maturity50150');

implied_volatility12 = wtiMarketVol;
maturity12 = wtiMaturity;
log_moneyness12 = wtiLogMoneyness;

% plot total implied volatility on 12

maturities12 = sort(unique(maturity12));
T12 = length(maturities12);


% fit SSVI to data
phifun = 'power_law';
parameters12 = fit_svi_surface(implied_volatility12, maturity12, log_moneyness12, phifun);

aaa = 1.0;


% get fitted total implied variances for 12
total_implied_variance12 = implied_volatility12.^2 .* maturity12;
model_total_implied_variance12 = zeros(size(total_implied_variance12));
model_implied_volatility12 = zeros(size(total_implied_variance12));

for t = 1:T12
    pos = maturity12==maturities12(t);
    [model_total_implied_variance12(pos), model_implied_volatility12(pos)] = ...
        svi_jumpwing(log_moneyness12(pos), parameters12(:,t), maturities12(t));
end

% reshape into matrix
implied_volatility12_matrix = reshape(implied_volatility12,[7 9]);
model_implied_volatility12_matrix = reshape(model_implied_volatility12,[7 9]);



implied_volatility13 = wtiMarketVol13;
maturity13 = wtiMaturity13;
log_moneyness13 = wtiLogMoneyness13;

% parameters calibrated on 12 predicted on 13

total_implied_variance13 = implied_volatility13.^2 .* maturity13;
model_total_implied_variance12_13 = zeros(size(total_implied_variance13));
model_implied_volatility12_13 = zeros(size(total_implied_variance13));

% parameters calibrated on 13
model_total_implied_variance13 = zeros(size(total_implied_variance13));
model_implied_volatility13 = zeros(size(total_implied_variance13));


maturities13 = sort(unique(maturity13));
T13 = length(maturities13);



% fit SSVI to data on 13
phifun = 'power_law';
parameters13 = fit_svi_surface(implied_volatility13, maturity13, log_moneyness13, phifun);


for t = 1:T13
    pos = maturity13==maturities13(t);
    [model_total_implied_variance13(pos), model_implied_volatility13(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters13(:,t), maturities13(t));
end

%% 12_13vol : 12 params && 13 atmvol only
parameters12_13vol = parameters12;
parameters12_13vol_up = parameters12;
parameters12_13vol_down = parameters12;
% update atm vol only
for i=1:T13
    parameters12_13vol(1,i) = parameters13(1,i);
    parameters12_13vol_up(1,i) = (sqrt(parameters13(1,i)) + 0.01)^2;
    parameters12_13vol_down(1,i) = (sqrt(parameters13(1,i)) - 0.01)^2;
end

model_total_implied_variance12_13vol = zeros(size(total_implied_variance13));
model_implied_volatility12_13vol = zeros(size(total_implied_variance13));

model_total_implied_variance12_13vol_up = zeros(size(total_implied_variance13));
model_implied_volatility12_13vol_up = zeros(size(total_implied_variance13));

model_total_implied_variance12_13vol_down = zeros(size(total_implied_variance13));
model_implied_volatility12_13vol_down = zeros(size(total_implied_variance13));

for t = 1:T13
    pos = maturity13==maturities13(t);
    [model_total_implied_variance12_13vol(pos), model_implied_volatility12_13vol(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12_13vol(:,t), maturities13(t));
    
    [model_total_implied_variance12_13vol_up(pos), model_implied_volatility12_13vol_up(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12_13vol_up(:,t), maturities13(t));
    
    [model_total_implied_variance12_13vol_down(pos), model_implied_volatility12_13vol_down(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12_13vol_down(:,t), maturities13(t));
end

model_implied_volatility12_13vol_matrix = reshape(model_implied_volatility12_13vol,[7 9]);
model_implied_volatility12_13vol_up_matrix = reshape(model_implied_volatility12_13vol_up,[7 9]);
model_implied_volatility12_13vol_down_matrix = reshape(model_implied_volatility12_13vol_down,[7 9]);

%% 12_13vol : 12 params && 13 atmvol && skew only
parameters12_13vol_skew = parameters12;
parameters12_13vol_skew_up = parameters12;
parameters12_13vol_skew_down = parameters12;

% update atm vol only
for i=1:T13
    %vol and skew only
    parameters12_13vol_skew(1,i) = parameters13(1,i);
    parameters12_13vol_skew(2,i) = parameters13(2,i);
    
    parameters12_13vol_skew_up(1,i) = parameters13(1,i);
    parameters12_13vol_skew_up(2,i) = parameters13(2,i) + 0.01;
    
    parameters12_13vol_skew_down(1,i) = parameters13(1,i);
    parameters12_13vol_skew_down(2,i) = parameters13(2,i) - 0.01;
end

model_total_implied_variance12_13vol_skew = zeros(size(total_implied_variance13));
model_implied_volatility12_13vol_skew = zeros(size(total_implied_variance13));

model_total_implied_variance12_13vol_skew_up = zeros(size(total_implied_variance13));
model_implied_volatility12_13vol_skew_up = zeros(size(total_implied_variance13));

model_total_implied_variance12_13vol_skew_down = zeros(size(total_implied_variance13));
model_implied_volatility12_13vol_skew_down = zeros(size(total_implied_variance13));

for t = 1:T13
    pos = maturity13==maturities13(t);
    [model_total_implied_variance12_13vol_skew(pos), model_implied_volatility12_13vol_skew(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12_13vol_skew(:,t), maturities13(t));
    
    [model_total_implied_variance12_13vol_skew_up(pos), model_implied_volatility12_13vol_skew_up(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12_13vol_skew_up(:,t), maturities13(t));
    
    [model_total_implied_variance12_13vol_skew_down(pos), model_implied_volatility12_13vol_skew_down(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12_13vol_skew_down(:,t), maturities13(t));
end

model_implied_volatility12_13vol_skew_matrix = reshape(model_implied_volatility12_13vol_skew,[7 9]);
model_implied_volatility12_13vol_skew_up_matrix = reshape(model_implied_volatility12_13vol_skew_up,[7 9]);
model_implied_volatility12_13vol_skew_down_matrix = reshape(model_implied_volatility12_13vol_skew_down,[7 9]);

%% fit SSVI to data on 12_13vol_skew interpolated on 50~150
phifun = 'power_law';

parameters12_13vol_skew_50150 = fit_svi_surface(wtiMarketVol50150, maturity50150, wtiLogMoneyness50150, phifun);

total_implied_variance50150 = wtiMarketVol50150.^2 .* maturity50150;
model_total_implied_variance12_13vol_skew_50150 = zeros(size(total_implied_variance50150));
model_implied_volatility12_13vol_skew_50150 = zeros(size(total_implied_variance50150));

for t = 1:T13
    pos = maturity50150==maturities13(t);
    [model_total_implied_variance12_13vol_skew_50150(pos), model_implied_volatility12_13vol_skew_50150(pos)] = ...
        svi_jumpwing(wtiLogMoneyness50150(pos), parameters12_13vol_skew_50150(:,t), maturities13(t));
end

model_implied_volatility12_13vol_skew_50150_matrix = reshape(model_implied_volatility12_13vol_skew_50150,[7 11]);

%% 12_13 : params 12

for t = 1:T13
    pos = maturity13==maturities13(t);
    [model_total_implied_variance12_13(pos), model_implied_volatility12_13(pos)] = ...
        svi_jumpwing(log_moneyness13(pos), parameters12(:,t), maturities13(t));
end

% reshape into matrix
implied_volatility13_matrix = reshape(implied_volatility13,[7 9]);
model_implied_volatility13_matrix = reshape(model_implied_volatility13,[7 9]);

model_implied_volatility12_13_matrix = reshape(model_implied_volatility12_13,[7 9]);

aaa = 1.0;





In [ ]:
%example_WTI.m

% % The Stochastic Volatility Inspired parameterization of the Implied
% % Volatility Smile
% %% raw SVI parameterization
% a = -0.041;
% b = 0.1331;
% m = 0.3586;
% rho = 0.3060;
% sigma = 0.4153;
% param_raw = [a; b; m; rho; sigma];
% 
% k = -1.5:0.01:1.5;
% w_r = svi_raw(k,param_raw);
% 
% plot(k,w_r, 'ob');
% 
% %% natural SVI parameterization
% 
% param_natural = svi_convertparameters( param_raw, 'raw', 'natural');
% w_n = svi_natural(k,param_natural);
% 
% plot(k,w_r, 'ob');
% hold on
% plot(k,w_n, 'xr');
% legend('Raw parameterization','Natural parameterization');
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_n-w_r)>1e-10));
% 
% %% Jump-Wings parameterization
% 
% tau = 1;
% param_jw = svi_convertparameters( param_raw, 'raw', 'jumpwing', tau);
% w_j = svi_jumpwing(k,param_jw, tau);
% 
% plot(k,w_r, 'ob');
% hold on
% plot(k,w_j, 'xr');
% legend('Raw parameterization','Jump-Wing parameterization');
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_j-w_r)>1e-10));
% 
% %% Test conversion in other direction
% v = 0.01742625;
% psi = 0.1752111;
% p = 0.6997381;
% c = 1.316798;
% vt = 0.0116249;
% param_jw = [v; psi; p; c; vt];
% param_raw = svi_convertparameters( param_jw, 'jumpwing', 'raw', tau);
% param_natural = svi_convertparameters( param_jw, 'jumpwing', 'natural', tau);
% 
% w_r = svi_raw(k,param_raw);
% w_n = svi_natural(k,param_natural);
% w_j = svi_jumpwing(k,param_jw, tau);
% 
% plot(k, w_r, 'ob')
% hold on
% plot(k, w_n, 'xr')
% plot(k, w_j, 'xg')
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_n-w_r)>1e-10));
% display(any(abs(w_j-w_r)>1e-10));
% display(any(abs(w_j-w_n)>1e-10));
% 
% %% Arbitrage-free Surface SVI
% theta = 0.1:0.1:1;
% rho = 0.3;
% phifun = 'heston_like';
% phi_param = (1+abs(rho))/4 + 1;
% 
% [totalvariance] = svi_surface(k, theta, rho, phifun, phi_param);
% plot(k, totalvariance)
% 
% phifun = @(theta, lambda) 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
% param_natural = [0; 0; rho; theta(4); phifun(theta(4),phi_param)];
% totalvariance_nat = svi_natural(k,param_natural);
% 
% plot(k, totalvariance_nat, 'ob')
% hold on
% plot(k, totalvariance(:,4), 'xr');
% hold off
% 
% display(any(abs(totalvariance_nat-totalvariance(:,4)) > 1e-10));
% 
% %% calibrate SSVI to Sample Data
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear put
% 
% % estimate implied volatility
% % implied_volatility_bid = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, bid);
% % implied_volatility_ask = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, ask);
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% 
% implied_volatility_bid = zeros(length(bid),1);
% implied_volatility_ask = zeros(length(bid),1);
% implied_volatility     = zeros(length(bid),1);
% 
% for idxhh =1:length(bid)
%     implied_volatility_bid(idxhh) = blackVolLBR(bid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
%     implied_volatility_ask(idxhh) = blackVolLBR(ask(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end

%% market data for WTI
load('wtiFutures.mat','wtiFutures');
load('wtiLogMoneyness.mat','wtiLogMoneyness');
load('wtiMarketVol.mat','wtiMarketVol');
load('wtiMaturity.mat','wtiMaturity');
load('wtiStrikes.mat','wtiStrikes');

implied_volatility = wtiMarketVol;
maturity = wtiMaturity;
log_moneyness = wtiLogMoneyness;


% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

% plot fitted data versus market data
for t = 1:T
    pos = maturity==maturities(t);
    
    subplot(3,5,t);
    plot(log_moneyness(pos), implied_volatility(pos), 'xb');
    hold on
    plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
    hold off
end

aaa = 1.0;

%% inter-/ extrapolation of SVI fit
clear
load sampledata
% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
forward = close.*exp((interest_rate-dividend_yield).*maturity);
pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
ask = ask(~pos_remove);
bid = bid(~pos_remove);
dividend_yield = dividend_yield(~pos_remove);
forward = forward(~pos_remove);
interest_rate = interest_rate(~pos_remove);
maturity = maturity(~pos_remove);
put = put(~pos_remove);
strike = strike(~pos_remove);
clear close pos_remove

% prepare data
bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
mid = (bid+ask)/2;
log_moneyness = log(strike./forward);
clear ask bid put

% estimate implied volatility
% implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
end

% fit SSVI to data
phifun = 'power_law';
[parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% load('SVIparameters.mat','parameters');
% load('SVItheta.mat','theta');
% load('SVImaturities.mat','maturities');


log_moneyness = -1.5:0.1:1.5;

[~, idx] = ismember(maturities, maturity);
forward_theta = forward(idx);
interestrate_theta = interest_rate(idx);

tau_interp = 30/365.25;
forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');


[call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
    forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);

strike = forward_theta*exp(log_moneyness);
figure
plot(strike, call_price);



In [ ]:
% Example IPCA estimation code
%
% What follows shows how to use num_IPCA_estimate_als.m within a numerical iterative
% procedure.
% 
% Assume the data you are factoring is Y -- let it be a NxT array. The instrumenting data
% is Z -- let it be NxLxT. T is the time dimension. N is the (maximum) cross-sectional
% dimension. L is the number of different instruments being used. Assume that any proper
% lagging is already reflected in Y(:,t) and Z(:,:,t); for example, in a APT application
% we'd have time t returns in Y(:,t) and time t-1 instruments in Z(:,:,t).
%
% We might not see all N observations every time period -- some might be missing.
% Observations are indexed by (n,t). Observation (n,t) is missing if any element of Y(n,t)
% or Z(n,:,t) is missing (NaN). IPCA can still be estimated. Calculate LOC as the NxT
% logical array where LOC(n,t)=true is observation (n,t) is observed, and is
% LOC(n,t)=false otherwise. Let Nts=sum(LOC); it is the T-vector of the number of
% non-missing observations each time period.
%
% Define W(:,:,t) = Z(LOC(:,t),:,t)'*Z(LOC(:,t),:,t)/Nts(t). Hence, W is a LxLxT array.
%
% Define X(:,t) = Z(LOC(:,t),:,t)'*Y(LOC(:,t),t)/Nts(t). Hence X is a LxT array.
%
% IPCA estimation requires
% 1. W
% 2. X
% 3. Nts
% 4. K (the chosen number of factors -- must be weakly less than L)
% 5. Initial guesses for Gamma and Factor
% 
% In this example, the initial guesses for Gamma and Factor are taken from the SVD of X;
% but other initializations could be used. One generally feels more confident that a
% global optimum has been achieved by IPCA if one starts the numerical procedure from
% various initial guesses and ends at the same place
%
% The tolerance and maximum number of iterations can be adjusted.
% load('fundamentalData', 'tday', 'syms', 'mid');
clc;
clear;
load('fundamentalData20220815', 'tday', 'syms', 'mid');
lookback=252; % minimum number of observations required for PCA
% lookback= 60; % minimum number of observations required for PCA
window = 60;
t = lookback+1;
trainset=t-window+1:t;

ret1=calculateReturns(mid, 1);
R=ret1(trainset, :);
hasData=find(all(isfinite(R), 1)); % avoid any stocks with missing returns
R=R(:, hasData);
X= R';

%% Numerical choices
MaxIterations       = 10000;
Tolerance           = 1e-6;
K=5;
%% Initial guess
[Gamma_Old,s,v]     = svds(X,K);
Factor_Old          = s*v';

%% Algorithm
tol         = 1;
iter        = 0;
while (iter <= MaxIterations) && (tol > Tolerance)
    % Run regressions using Old estimates
    [Gamma_New,Factor_New] = num_IPCA_estimate_ALS(Gamma_Old,W,X,Nts);
    % Calculate change in Old and New estimates
    tol         = max([ abs(Gamma_New(:)-Gamma_Old(:)) ; abs(Factor_New(:)-Factor_Old(:)) ]); % other convergence norms could be used
    % Replace Old estimates for the next iteration
    Factor_Old  = Factor_New;
    Gamma_Old   = Gamma_New;
    iter        = iter+1;
end
Gamma   = Gamma_New;
Factor  = Factor_New;



In [ ]:
%example_Brent_orig20181206_new.m


%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

% load('WTIForward.mat','WTIForward')
% load('WTIexpiry.mat','WTIexpiry')
% load('WTIPremiums.mat','WTIPremiums')
% load('WTIStrikes.mat','WTIStrikes')
% load('WTIrate.mat','WTIrate')

% load('brentPremiums.mat','brentPremiums')
% load('brentPremiums1205.mat','brentPremiums1205')

xlsRawData1206 = xlsread('xlsMarketDataBrent_1206_new.xlsx');
brentRaw1206 = xlsRawData1206(4:end,2:end);
brentPremium1206 = brentRaw1206(:);
mid = brentPremium1206;

valueDate = '2018-12-06';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-01-28'),H_Date('2019-02-25'),H_Date('2019-04-25'),H_Date('2019-10-28'),...
                      H_Date('2020-04-27'),H_Date('2020-10-27')};

optionMaturity = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity)
    optionMaturity(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity6 = optionMaturity/365.0;
maturityLong = repelem(maturity6,55)';

strike6 = [21:1:75];
strikeLong = repmat(strike6,1,6)';
forward6 = [60.33	60.55	60.99	61.06	61.14	61.02];
rate6 = [0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];
rateLong = repelem(rate6,55)';
forwardLong = repelem(forward6,55)';

maturity = maturityLong;
strike = strikeLong;
rate = rateLong;
forward = forwardLong;

pos_remove = mid < 0.02 |  isnan(mid);

% pos_remove = isnan(WTIPremiums);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = rate(~pos_remove);
strike = strike(~pos_remove);
maturity = maturity(~pos_remove);
% clear close pos_remove

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
bloombergForwardMoneyness = [0.787982761	0.83769269	0.871970827	0.900580143	0.926255594	0.949825957	0.97188795	0.992922261	1.013194099	1.032985248	1.052610641	1.072434941	1.092971987	1.114868225	1.139184485	1.167893254	1.205900879
0.763748968	0.817324525	0.855293146	0.887497936	0.916911643	0.94416185	0.969876135	0.994533443	1.018513625	1.042146986	1.065763832	1.089810074	1.114814203	1.141486375	1.17104872	1.206193229	1.253526012
0.733103788	0.789719626	0.832693884	0.870372192	0.905115593	0.937579931	0.968486637	0.998458764	1.027955403	1.057386457	1.08724381	1.118019347	1.150336121	1.184964748	1.223430071	1.269175275	1.330185276
0.676744186	0.741991484	0.794792008	0.842302653	0.887045529	0.929741238	0.971143138	1.011742548	1.051899771	1.092171634	1.13322961	1.175712414	1.220258762	1.26799869	1.321208647	1.384081232	1.466557484
0.655986261	0.724762839	0.781599607	0.832940792	0.881648675	0.928573765	0.974419365	1.01944717	1.063689892	1.107605496	1.151864573	1.197170429	1.244471704	1.295600262	1.353647367	1.424370298	1.520821066
0.640560472	0.712143559	0.772271386	0.826876434	0.878613569	0.928482465	0.977171419	1.024844313	1.071566699	1.11791216	1.164716486	1.212700754	1.262962963	1.317535234	1.380088496	1.457309079	1.56409374
];

bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));

for t=1:T
    params.idx = t;
    for idxhh=1:length(bb_forwardMoneyness1)
        bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
        bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
    end
end


cc = 0.0;
aa =  findTargetStrike(-0.1,params);
bb = 0;
function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end


% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% 
% bbb= 0.0;
% 
% %% market data for WTI
% load('wtiFutures.mat','wtiFutures');
% load('wtiLogMoneyness.mat','wtiLogMoneyness');
% load('wtiMarketVol.mat','wtiMarketVol');
% load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');
% 
% implied_volatility = wtiMarketVol;
% maturity = wtiMaturity;
% log_moneyness = wtiLogMoneyness;
% 
% 
% % plot total implied volatility
% total_implied_volatility = implied_volatility.^2 .* maturity;
% maturities = sort(unique(maturity));
% T = length(maturities);
% figure
% hold on
% for t = 1:T
%     pos = maturity==maturities(t);
%     [x, idx] = sort(log_moneyness(pos));
%     y = total_implied_volatility(pos);
%     y = y(idx);
%     plot(x,y);
% end
% hold off
% 
% % fit SSVI to data
% phifun = 'power_law';
% parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % get fitted total implied variances
% total_implied_variance = implied_volatility.^2 .* maturity;
% model_total_implied_variance = zeros(size(total_implied_variance));
% model_implied_volatility = zeros(size(total_implied_variance));
% for t = 1:T
%     pos = maturity==maturities(t);
%     [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
%         svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
% end
% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% aaa = 1.0;
% 
% %% inter-/ extrapolation of SVI fit
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear ask bid put
% 
% % estimate implied volatility
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% % implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
% for idxhh =1:length(mid)
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end
% 
% % fit SSVI to data
% phifun = 'power_law';
% [parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % load('SVIparameters.mat','parameters');
% % load('SVItheta.mat','theta');
% % load('SVImaturities.mat','maturities');
% 
% 
% log_moneyness = -1.5:0.1:1.5;
% 
% [~, idx] = ismember(maturities, maturity);
% forward_theta = forward(idx);
% interestrate_theta = interest_rate(idx);
% 
% tau_interp = 30/365.25;
% forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
% interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');
% 
% 
% [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
%     forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);
% 
% strike = forward_theta*exp(log_moneyness);
% figure
% plot(strike, call_price);



In [ ]:
%example_Brent_orig20181206.m

% % The Stochastic Volatility Inspired parameterization of the Implied
% % Volatility Smile
% %% raw SVI parameterization
% a = -0.041;
% b = 0.1331;
% m = 0.3586;
% rho = 0.3060;
% sigma = 0.4153;
% param_raw = [a; b; m; rho; sigma];
% 
% k = -1.5:0.01:1.5;
% w_r = svi_raw(k,param_raw);
% 
% plot(k,w_r, 'ob');
% 
% %% natural SVI parameterization
% 
% param_natural = svi_convertparameters( param_raw, 'raw', 'natural');
% w_n = svi_natural(k,param_natural);
% 
% plot(k,w_r, 'ob');
% hold on
% plot(k,w_n, 'xr');
% legend('Raw parameterization','Natural parameterization');
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_n-w_r)>1e-10));
% 
% %% Jump-Wings parameterization
% 
% tau = 1;
% param_jw = svi_convertparameters( param_raw, 'raw', 'jumpwing', tau);
% w_j = svi_jumpwing(k,param_jw, tau);
% 
% plot(k,w_r, 'ob');
% hold on
% plot(k,w_j, 'xr');
% legend('Raw parameterization','Jump-Wing parameterization');
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_j-w_r)>1e-10));
% 
% %% Test conversion in other direction
% v = 0.01742625;
% psi = 0.1752111;
% p = 0.6997381;
% c = 1.316798;
% vt = 0.0116249;
% param_jw = [v; psi; p; c; vt];
% param_raw = svi_convertparameters( param_jw, 'jumpwing', 'raw', tau);
% param_natural = svi_convertparameters( param_jw, 'jumpwing', 'natural', tau);
% 
% w_r = svi_raw(k,param_raw);
% w_n = svi_natural(k,param_natural);
% w_j = svi_jumpwing(k,param_jw, tau);
% 
% plot(k, w_r, 'ob')
% hold on
% plot(k, w_n, 'xr')
% plot(k, w_j, 'xg')
% hold off
% 
% % check that results are equivalent
% display(any(abs(w_n-w_r)>1e-10));
% display(any(abs(w_j-w_r)>1e-10));
% display(any(abs(w_j-w_n)>1e-10));
% 
% %% Arbitrage-free Surface SVI
% theta = 0.1:0.1:1;
% rho = 0.3;
% phifun = 'heston_like';
% phi_param = (1+abs(rho))/4 + 1;
% 
% [totalvariance] = svi_surface(k, theta, rho, phifun, phi_param);
% plot(k, totalvariance)
% 
% phifun = @(theta, lambda) 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
% param_natural = [0; 0; rho; theta(4); phifun(theta(4),phi_param)];
% totalvariance_nat = svi_natural(k,param_natural);
% 
% plot(k, totalvariance_nat, 'ob')
% hold on
% plot(k, totalvariance(:,4), 'xr');
% hold off
% 
% display(any(abs(totalvariance_nat-totalvariance(:,4)) > 1e-10));

%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

% load('WTIForward.mat','WTIForward')
% load('WTIexpiry.mat','WTIexpiry')
% load('WTIPremiums.mat','WTIPremiums')
load('WTIStrikes.mat','WTIStrikes')
load('WTIrate.mat','WTIrate')

load('brentPremiums.mat','brentPremiums')
valueDate = '2018-12-06';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-01-28'),H_Date('2019-02-25'),H_Date('2019-05-28'),H_Date('2019-11-26'),...
                      H_Date('2020-05-26'),H_Date('2020-11-25')};

optionMaturity = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity)
    optionMaturity(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity6 = optionMaturity/365.0;
maturityLong = repelem(maturity6,55)';

strike6 = [];
forward6 = [60.33	60.55	60.99	61.06	61.14	61.02];
rate6 = [0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];
reateLong = repelem(rate6,55)';
forwardLong = repelem(forward6,55)';
% premium = 0.0;

% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% mid = WTIPremiums;
mid = brentPremiums;

forward = forwardLong;
maturity = maturityLong;

pos_remove = mid < 0.02 |  isnan(mid);

% pos_remove = isnan(WTIPremiums);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = WTIrate(~pos_remove);
strike = WTIStrikes(~pos_remove);
maturity = maturity(~pos_remove);
% clear close pos_remove

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
bloombergForwardMoneyness = [0.770411776	0.855897149	0.917477175	0.969945966	1.018203838	1.06497112	1.113322154	1.169032979	1.249506242
0.748552876	0.841539889	0.90987013	0.968831169	1.023692022	1.07755102	1.133988868	1.199721707	1.295454545
0.720305596	0.822772459	0.900257732	0.968464654	1.033081738	1.097790869	1.166439617	1.245360825	1.362242268
0.671096892	0.791389397	0.886032907	0.971517367	1.053345521	1.135886654	1.224771481	1.328957952	1.485411335
0.649012075	0.777296012	0.880717161	0.974698134	1.06381266	1.152140505	1.246121478	1.359220637	1.553311379
0.633632533	0.767594798	0.877614948	0.97713867	1.070727239	1.163418208	1.263454845	1.387378641	1.592855834
];

bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));

for t=1:T
    params.idx = t;
    for idxhh=1:length(bb_forwardMoneyness1)
        bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
        bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
    end
end


cc = 0.0;
aa =  findTargetStrike(-0.1,params);
bb = 0;
function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end


% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% 
% bbb= 0.0;
% 
% %% market data for WTI
% load('wtiFutures.mat','wtiFutures');
% load('wtiLogMoneyness.mat','wtiLogMoneyness');
% load('wtiMarketVol.mat','wtiMarketVol');
% load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');
% 
% implied_volatility = wtiMarketVol;
% maturity = wtiMaturity;
% log_moneyness = wtiLogMoneyness;
% 
% 
% % plot total implied volatility
% total_implied_volatility = implied_volatility.^2 .* maturity;
% maturities = sort(unique(maturity));
% T = length(maturities);
% figure
% hold on
% for t = 1:T
%     pos = maturity==maturities(t);
%     [x, idx] = sort(log_moneyness(pos));
%     y = total_implied_volatility(pos);
%     y = y(idx);
%     plot(x,y);
% end
% hold off
% 
% % fit SSVI to data
% phifun = 'power_law';
% parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % get fitted total implied variances
% total_implied_variance = implied_volatility.^2 .* maturity;
% model_total_implied_variance = zeros(size(total_implied_variance));
% model_implied_volatility = zeros(size(total_implied_variance));
% for t = 1:T
%     pos = maturity==maturities(t);
%     [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
%         svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
% end
% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% aaa = 1.0;
% 
% %% inter-/ extrapolation of SVI fit
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear ask bid put
% 
% % estimate implied volatility
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% % implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
% for idxhh =1:length(mid)
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end
% 
% % fit SSVI to data
% phifun = 'power_law';
% [parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % load('SVIparameters.mat','parameters');
% % load('SVItheta.mat','theta');
% % load('SVImaturities.mat','maturities');
% 
% 
% log_moneyness = -1.5:0.1:1.5;
% 
% [~, idx] = ismember(maturities, maturity);
% forward_theta = forward(idx);
% interestrate_theta = interest_rate(idx);
% 
% tau_interp = 30/365.25;
% forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
% interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');
% 
% 
% [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
%     forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);
% 
% strike = forward_theta*exp(log_moneyness);
% figure
% plot(strike, call_price);



In [ ]:
%example_Brent_orig20181205_new.m

%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

% load('WTIForward.mat','WTIForward')
% load('WTIexpiry.mat','WTIexpiry')
% load('WTIPremiums.mat','WTIPremiums')
% load('WTIStrikes.mat','WTIStrikes')
% load('WTIrate.mat','WTIrate')

% load('brentPremiums.mat','brentPremiums')
% load('brentPremiums1205.mat','brentPremiums1205')

% xlsRawData1206 = xlsread('xlsMarketDataBrent_1206_new.xlsx');
xlsRawData1206 = xlsread('xlsMarketDataBrent_1205_new.xlsx');

brentRaw1206 = xlsRawData1206(4:end,2:end);
brentPremium1206 = brentRaw1206(:);
mid = brentPremium1206;

valueDate = '2018-12-05';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-01-28'),H_Date('2019-02-25'),H_Date('2019-04-25'),H_Date('2019-10-28'),...
                      H_Date('2020-04-27'),H_Date('2020-10-27')};

optionMaturity = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity)
    optionMaturity(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity6 = optionMaturity/365.0;
maturityLong = repelem(maturity6,55)';

strike6 = [21:1:75];
strikeLong = repmat(strike6,1,6)';
forward6 = [61.76	61.94	62.32	62.29	62.29	62.14];
rate6 = [0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];
rateLong = repelem(rate6,55)';
forwardLong = repelem(forward6,55)';

maturity = maturityLong;
strike = strikeLong;
rate = rateLong;
forward = forwardLong;

pos_remove = mid < 0.02 |  isnan(mid);

% pos_remove = isnan(WTIPremiums);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = rate(~pos_remove);
strike = strike(~pos_remove);
maturity = maturity(~pos_remove);
% clear close pos_remove

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
bloombergForwardMoneyness = [0.787982761	0.83769269	0.871970827	0.900580143	0.926255594	0.949825957	0.97188795	0.992922261	1.013194099	1.032985248	1.052610641	1.072434941	1.092971987	1.114868225	1.139184485	1.167893254	1.205900879
0.763748968	0.817324525	0.855293146	0.887497936	0.916911643	0.94416185	0.969876135	0.994533443	1.018513625	1.042146986	1.065763832	1.089810074	1.114814203	1.141486375	1.17104872	1.206193229	1.253526012
0.733103788	0.789719626	0.832693884	0.870372192	0.905115593	0.937579931	0.968486637	0.998458764	1.027955403	1.057386457	1.08724381	1.118019347	1.150336121	1.184964748	1.223430071	1.269175275	1.330185276
0.676744186	0.741991484	0.794792008	0.842302653	0.887045529	0.929741238	0.971143138	1.011742548	1.051899771	1.092171634	1.13322961	1.175712414	1.220258762	1.26799869	1.321208647	1.384081232	1.466557484
0.655986261	0.724762839	0.781599607	0.832940792	0.881648675	0.928573765	0.974419365	1.01944717	1.063689892	1.107605496	1.151864573	1.197170429	1.244471704	1.295600262	1.353647367	1.424370298	1.520821066
0.640560472	0.712143559	0.772271386	0.826876434	0.878613569	0.928482465	0.977171419	1.024844313	1.071566699	1.11791216	1.164716486	1.212700754	1.262962963	1.317535234	1.380088496	1.457309079	1.56409374
];

bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));

for t=1:T
    params.idx = t;
    for idxhh=1:length(bb_forwardMoneyness1)
        bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
        bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
    end
end


cc = 0.0;
aa =  findTargetStrike(-0.1,params);
bb = 0;
function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end


% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% 
% bbb= 0.0;
% 
% %% market data for WTI
% load('wtiFutures.mat','wtiFutures');
% load('wtiLogMoneyness.mat','wtiLogMoneyness');
% load('wtiMarketVol.mat','wtiMarketVol');
% load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');
% 
% implied_volatility = wtiMarketVol;
% maturity = wtiMaturity;
% log_moneyness = wtiLogMoneyness;
% 
% 
% % plot total implied volatility
% total_implied_volatility = implied_volatility.^2 .* maturity;
% maturities = sort(unique(maturity));
% T = length(maturities);
% figure
% hold on
% for t = 1:T
%     pos = maturity==maturities(t);
%     [x, idx] = sort(log_moneyness(pos));
%     y = total_implied_volatility(pos);
%     y = y(idx);
%     plot(x,y);
% end
% hold off
% 
% % fit SSVI to data
% phifun = 'power_law';
% parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % get fitted total implied variances
% total_implied_variance = implied_volatility.^2 .* maturity;
% model_total_implied_variance = zeros(size(total_implied_variance));
% model_implied_volatility = zeros(size(total_implied_variance));
% for t = 1:T
%     pos = maturity==maturities(t);
%     [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
%         svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
% end
% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% aaa = 1.0;
% 
% %% inter-/ extrapolation of SVI fit
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear ask bid put
% 
% % estimate implied volatility
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% % implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
% for idxhh =1:length(mid)
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end
% 
% % fit SSVI to data
% phifun = 'power_law';
% [parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % load('SVIparameters.mat','parameters');
% % load('SVItheta.mat','theta');
% % load('SVImaturities.mat','maturities');
% 
% 
% log_moneyness = -1.5:0.1:1.5;
% 
% [~, idx] = ismember(maturities, maturity);
% forward_theta = forward(idx);
% interestrate_theta = interest_rate(idx);
% 
% tau_interp = 30/365.25;
% forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
% interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');
% 
% 
% [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
%     forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);
% 
% strike = forward_theta*exp(log_moneyness);
% figure
% plot(strike, call_price);



In [ ]:
%example_Brent_orig20181205.m

%% calibrate SSVI to Sample Data
clc
clear
% load sampledata

% load('WTIForward.mat','WTIForward')
% load('WTIexpiry.mat','WTIexpiry')
% load('WTIPremiums.mat','WTIPremiums')
% load('WTIStrikes.mat','WTIStrikes')
% load('WTIrate.mat','WTIrate')

% load('brentPremiums.mat','brentPremiums')
% load('brentPremiums1205.mat','brentPremiums1205')

xlsRawData1205 = xlsread('xlsMarketDataBrent_1205.xlsx');
brentRaw1205 = xlsRawData1205(4:end,2:end);
brentPremium1205 = brentRaw1205(:);
mid = brentPremium1205;

valueDate = '2018-12-05';
valueDateH = H_Date(valueDate);
optionMaturityDate = {H_Date('2019-01-28'),H_Date('2019-02-25'),H_Date('2019-04-25'),H_Date('2019-10-28'),...
                      H_Date('2020-04-27'),H_Date('2020-10-27')};

optionMaturity = zeros(1,length(optionMaturityDate));
for idxhh=1:length(optionMaturity)
    optionMaturity(idxhh) = DateDiff(optionMaturityDate{idxhh},valueDateH);
end

maturity6 = optionMaturity/365.0;
maturityLong = repelem(maturity6,55)';

strike6 = [21:1:75];
strikeLong = repmat(strike6,1,6)';
forward6 = [61.76	61.94	62.32	62.29	62.29	62.14];
rate6 = [0.02209	0.02288	0.02392	0.0252	0.02601	0.0264];
rateLong = repelem(rate6,55)';
forwardLong = repelem(forward6,55)';

maturity = maturityLong;
strike = strikeLong;
rate = rateLong;
forward = forwardLong;

pos_remove = mid < 0.02 |  isnan(mid);

% pos_remove = isnan(WTIPremiums);
mid = mid(~pos_remove);
forward = forward(~pos_remove);
rate = rate(~pos_remove);
strike = strike(~pos_remove);
maturity = maturity(~pos_remove);
% clear close pos_remove

% prepare data
log_moneyness = log(strike./forward);

implied_volatility     = zeros(length(mid),1);

for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), -1);
end

aaa = 0.0;
% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

deltaMoneynessP = [-0.1:-0.05:-0.45]';
deltaMoneynessC = [0.5:-0.05:0.1]';
deltaMoneyness = [deltaMoneynessP;deltaMoneynessC];

deltaMoneyness_implied_volatility = zeros(length(maturities),length(deltaMoneyness));
deltaMoneyness_to_forwardMoneyness = zeros(length(maturities),length(deltaMoneyness));

params.parameters = parameters;
params.maturities = maturities;
params.idx = 1;

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness)
        deltaMoneyness_to_forwardMoneyness(t,idxhh) = findTargetStrike(deltaMoneyness(idxhh),params);
        deltaMoneyness_implied_volatility(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness(t,idxhh),params);
    end
end

% deltaMoneyness -0.1 -0.2 ...
deltaMoneynessP1 = [-0.1:-0.1:-0.4]';
deltaMoneynessC1 = [0.5:-0.1:0.1]';
deltaMoneyness1 = [deltaMoneynessP1;deltaMoneynessC1];

deltaMoneyness_implied_volatility1 = zeros(length(maturities),length(deltaMoneyness1));
deltaMoneyness_to_forwardMoneyness1 = zeros(length(maturities),length(deltaMoneyness1));

for t=1:T
    params.idx = t;
    for idxhh=1:length(deltaMoneyness1)
        deltaMoneyness_to_forwardMoneyness1(t,idxhh) = findTargetStrike(deltaMoneyness1(idxhh),params);
        deltaMoneyness_implied_volatility1(t,idxhh) = simple_svi_jumpwing(deltaMoneyness_to_forwardMoneyness1(t,idxhh),params);
    end
end

%% target forward moneyness
bloombergForwardMoneyness = [0.770411776	0.855897149	0.917477175	0.969945966	1.018203838	1.06497112	1.113322154	1.169032979	1.249506242
0.748552876	0.841539889	0.90987013	0.968831169	1.023692022	1.07755102	1.133988868	1.199721707	1.295454545
0.720305596	0.822772459	0.900257732	0.968464654	1.033081738	1.097790869	1.166439617	1.245360825	1.362242268
0.671096892	0.791389397	0.886032907	0.971517367	1.053345521	1.135886654	1.224771481	1.328957952	1.485411335
0.649012075	0.777296012	0.880717161	0.974698134	1.06381266	1.152140505	1.246121478	1.359220637	1.553311379
0.633632533	0.767594798	0.877614948	0.97713867	1.070727239	1.163418208	1.263454845	1.387378641	1.592855834
];

bb_implied_volatility1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));
bb_forwardMoneyness1 = zeros(length(maturities),size(bloombergForwardMoneyness,2));

for t=1:T
    params.idx = t;
    for idxhh=1:length(bb_forwardMoneyness1)
        bb_forwardMoneyness1(t,idxhh) = bloombergForwardMoneyness(t,idxhh) ;
        bb_implied_volatility1(t,idxhh) = simple_svi_jumpwing(bb_forwardMoneyness1(t,idxhh),params);
    end
end


cc = 0.0;
aa =  findTargetStrike(-0.1,params);
bb = 0;
function out = findTargetStrike(delta,params)
    if delta < 0
        p = norminv(-1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        
        targetFunc = @(x) std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    else
        p = norminv(1.0*delta);
        std1 = @(strike) simple_svi_jumpwing(strike,params)*sqrt(params.maturities(params.idx));
        targetFunc = @(x) -std1(x)*p + 0.5*std1(x)*std1(x)-log(x); 
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[0.1 2.0]);
        out = solutionK;
    end
end
function out = simple_svi_jumpwing(strike,params)
    parameters = params.parameters;
    maturities = params.maturities;
    idx = params.idx;
    
    [~, modelVol] = svi_jumpwing(log(strike), parameters(:,idx), maturities(idx));
    out = modelVol;
end


% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% 
% bbb= 0.0;
% 
% %% market data for WTI
% load('wtiFutures.mat','wtiFutures');
% load('wtiLogMoneyness.mat','wtiLogMoneyness');
% load('wtiMarketVol.mat','wtiMarketVol');
% load('wtiMaturity.mat','wtiMaturity');
% load('wtiStrikes.mat','wtiStrikes');
% 
% implied_volatility = wtiMarketVol;
% maturity = wtiMaturity;
% log_moneyness = wtiLogMoneyness;
% 
% 
% % plot total implied volatility
% total_implied_volatility = implied_volatility.^2 .* maturity;
% maturities = sort(unique(maturity));
% T = length(maturities);
% figure
% hold on
% for t = 1:T
%     pos = maturity==maturities(t);
%     [x, idx] = sort(log_moneyness(pos));
%     y = total_implied_volatility(pos);
%     y = y(idx);
%     plot(x,y);
% end
% hold off
% 
% % fit SSVI to data
% phifun = 'power_law';
% parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % get fitted total implied variances
% total_implied_variance = implied_volatility.^2 .* maturity;
% model_total_implied_variance = zeros(size(total_implied_variance));
% model_implied_volatility = zeros(size(total_implied_variance));
% for t = 1:T
%     pos = maturity==maturities(t);
%     [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
%         svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
% end
% 
% % plot fitted data versus market data
% for t = 1:T
%     pos = maturity==maturities(t);
%     
%     subplot(3,5,t);
%     plot(log_moneyness(pos), implied_volatility(pos), 'xb');
%     hold on
%     plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
%     hold off
% end
% 
% aaa = 1.0;
% 
% %% inter-/ extrapolation of SVI fit
% clear
% load sampledata
% % the sample data set consists of bid and ask prices of options written on
% % the S&P 500 index on 15-Sep-2011, the day before triple witching.
% 
% % remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
% forward = close.*exp((interest_rate-dividend_yield).*maturity);
% pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
% ask = ask(~pos_remove);
% bid = bid(~pos_remove);
% dividend_yield = dividend_yield(~pos_remove);
% forward = forward(~pos_remove);
% interest_rate = interest_rate(~pos_remove);
% maturity = maturity(~pos_remove);
% put = put(~pos_remove);
% strike = strike(~pos_remove);
% clear close pos_remove
% 
% % prepare data
% bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
% mid = (bid+ask)/2;
% log_moneyness = log(strike./forward);
% clear ask bid put
% 
% % estimate implied volatility
% % implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% % implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
% for idxhh =1:length(mid)
%     implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
% end
% 
% % fit SSVI to data
% phifun = 'power_law';
% [parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);
% 
% 
% % load('SVIparameters.mat','parameters');
% % load('SVItheta.mat','theta');
% % load('SVImaturities.mat','maturities');
% 
% 
% log_moneyness = -1.5:0.1:1.5;
% 
% [~, idx] = ismember(maturities, maturity);
% forward_theta = forward(idx);
% interestrate_theta = interest_rate(idx);
% 
% tau_interp = 30/365.25;
% forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
% interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');
% 
% 
% [call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
%     forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);
% 
% strike = forward_theta*exp(log_moneyness);
% figure
% plot(strike, call_price);



In [ ]:
% example.m

% The Stochastic Volatility Inspired parameterization of the Implied
% Volatility Smile
%% raw SVI parameterization
a = -0.041;
b = 0.1331;
m = 0.3586;
rho = 0.3060;
sigma = 0.4153;
param_raw = [a; b; m; rho; sigma];

k = -1.5:0.01:1.5;
w_r = svi_raw(k,param_raw);

plot(k,w_r, 'ob');

%% natural SVI parameterization

param_natural = svi_convertparameters( param_raw, 'raw', 'natural');
w_n = svi_natural(k,param_natural);

plot(k,w_r, 'ob');
hold on
plot(k,w_n, 'xr');
legend('Raw parameterization','Natural parameterization');
hold off

% check that results are equivalent
display(any(abs(w_n-w_r)>1e-10));

%% Jump-Wings parameterization

tau = 1;
param_jw = svi_convertparameters( param_raw, 'raw', 'jumpwing', tau);
w_j = svi_jumpwing(k,param_jw, tau);

plot(k,w_r, 'ob');
hold on
plot(k,w_j, 'xr');
legend('Raw parameterization','Jump-Wing parameterization');
hold off

% check that results are equivalent
display(any(abs(w_j-w_r)>1e-10));

%% Test conversion in other direction
v = 0.01742625;
psi = 0.1752111;
p = 0.6997381;
c = 1.316798;
vt = 0.0116249;
param_jw = [v; psi; p; c; vt];
param_raw = svi_convertparameters( param_jw, 'jumpwing', 'raw', tau);
param_natural = svi_convertparameters( param_jw, 'jumpwing', 'natural', tau);

w_r = svi_raw(k,param_raw);
w_n = svi_natural(k,param_natural);
w_j = svi_jumpwing(k,param_jw, tau);

plot(k, w_r, 'ob')
hold on
plot(k, w_n, 'xr')
plot(k, w_j, 'xg')
hold off

% check that results are equivalent
display(any(abs(w_n-w_r)>1e-10));
display(any(abs(w_j-w_r)>1e-10));
display(any(abs(w_j-w_n)>1e-10));

%% Arbitrage-free Surface SVI
theta = 0.1:0.1:1;
rho = 0.3;
phifun = 'heston_like';
phi_param = (1+abs(rho))/4 + 1;

[totalvariance] = svi_surface(k, theta, rho, phifun, phi_param);
plot(k, totalvariance)

phifun = @(theta, lambda) 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
param_natural = [0; 0; rho; theta(4); phifun(theta(4),phi_param)];
totalvariance_nat = svi_natural(k,param_natural);

plot(k, totalvariance_nat, 'ob')
hold on
plot(k, totalvariance(:,4), 'xr');
hold off

display(any(abs(totalvariance_nat-totalvariance(:,4)) > 1e-10));

%% calibrate SSVI to Sample Data
clear
load sampledata
% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
forward = close.*exp((interest_rate-dividend_yield).*maturity);
pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
ask = ask(~pos_remove);
bid = bid(~pos_remove);
dividend_yield = dividend_yield(~pos_remove);
forward = forward(~pos_remove);
interest_rate = interest_rate(~pos_remove);
maturity = maturity(~pos_remove);
put = put(~pos_remove);
strike = strike(~pos_remove);
clear close pos_remove

% prepare data
bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
mid = (bid+ask)/2;
log_moneyness = log(strike./forward);
clear put

% estimate implied volatility
% implied_volatility_bid = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, bid);
% implied_volatility_ask = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, ask);
% implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);

implied_volatility_bid = zeros(length(bid),1);
implied_volatility_ask = zeros(length(bid),1);
implied_volatility     = zeros(length(bid),1);

for idxhh =1:length(bid)
    implied_volatility_bid(idxhh) = blackVolLBR(bid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
    implied_volatility_ask(idxhh) = blackVolLBR(ask(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
end

% plot total implied volatility
total_implied_volatility = implied_volatility.^2 .* maturity;
maturities = sort(unique(maturity));
T = length(maturities);
figure
hold on
for t = 1:T
    pos = maturity==maturities(t);
    [x, idx] = sort(log_moneyness(pos));
    y = total_implied_volatility(pos);
    y = y(idx);
    plot(x,y);
end
hold off

% fit SSVI to data
phifun = 'power_law';
parameters = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% get fitted total implied variances
total_implied_variance = implied_volatility.^2 .* maturity;
model_total_implied_variance = zeros(size(total_implied_variance));
model_implied_volatility = zeros(size(total_implied_variance));
for t = 1:T
    pos = maturity==maturities(t);
    [model_total_implied_variance(pos), model_implied_volatility(pos)] = ...
        svi_jumpwing(log_moneyness(pos), parameters(:,t), maturities(t));
end

% plot fitted data versus market data
for t = 1:T
    pos = maturity==maturities(t);
    
    subplot(3,5,t);
    plot(log_moneyness(pos), implied_volatility_ask(pos), 'xb');
    hold on
    plot(log_moneyness(pos), implied_volatility_bid(pos), 'xb');
    plot(log_moneyness(pos), model_implied_volatility(pos), 'xr');
    hold off
end

%% inter-/ extrapolation of SVI fit
clear
load sampledata
% the sample data set consists of bid and ask prices of options written on
% the S&P 500 index on 15-Sep-2011, the day before triple witching.

% remove non-usable data: non-positive bid-ask spread, bid below 3/8, itm options
forward = close.*exp((interest_rate-dividend_yield).*maturity);
pos_remove = ask <= bid | bid < 3/8 | (strike < forward & put==0) | (forward <= strike & put==1);
ask = ask(~pos_remove);
bid = bid(~pos_remove);
dividend_yield = dividend_yield(~pos_remove);
forward = forward(~pos_remove);
interest_rate = interest_rate(~pos_remove);
maturity = maturity(~pos_remove);
put = put(~pos_remove);
strike = strike(~pos_remove);
clear close pos_remove

% prepare data
bid(put==1) = bid(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
ask(put==1) = ask(put==1) + exp(-interest_rate(put==1).*maturity(put==1)).*(forward(put==1)-strike(put==1));
mid = (bid+ask)/2;
log_moneyness = log(strike./forward);
clear ask bid put

% estimate implied volatility
% implied_volatility     = blsimpv(forward.*exp(-interest_rate.*maturity), strike, interest_rate, maturity, mid);
% implied_volatility     = blackVolLBR(mid.*exp(+interest_rate.*maturity),forward, strike, maturity, 1);
for idxhh =1:length(mid)
    implied_volatility(idxhh)     = blackVolLBR(mid(idxhh)*exp(+interest_rate(idxhh)*maturity(idxhh)),forward(idxhh), strike(idxhh), maturity(idxhh), 1);
end

% fit SSVI to data
phifun = 'power_law';
[parameters, theta, maturities] = fit_svi_surface(implied_volatility, maturity, log_moneyness, phifun);


% load('SVIparameters.mat','parameters');
% load('SVItheta.mat','theta');
% load('SVImaturities.mat','maturities');


log_moneyness = -1.5:0.1:1.5;

[~, idx] = ismember(maturities, maturity);
forward_theta = forward(idx);
interestrate_theta = interest_rate(idx);

tau_interp = 30/365.25;
forward_interp = interp1(maturities, forward_theta, tau_interp, 'linear');
interest_interp = interp1(zcb_days, zcb_rate, tau_interp*365.25, 'pchip');


[call_price, implied_volatility, total_implied_variance] = svi_interpolation(log_moneyness, tau_interp, ...
    forward_interp, interest_interp, parameters, theta, maturities, forward_theta, interestrate_theta);

strike = forward_theta*exp(log_moneyness);
figure
plot(strike, call_price);



In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
